<a href="https://colab.research.google.com/github/kashfay110/stock-market-python/blob/main/svr_ann_collab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [148]:
pip install mplfinance

In [149]:
# import quandl
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.datasets import make_classification
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import mplfinance as mpf
import keras
from keras.models import Sequential
from keras.layers import Dense
import os

In [150]:
# Get the stock data
#df = quandl.get("WIKI/FB")
#df = pd.read_csv (r'dataset/FB.csv')
# Take a look at data
#print(df.head())

pathname = r'AML.L.csv'

from google.colab import files
uploaded = files.upload()
import io
df = pd.read_csv(io.BytesIO(uploaded[pathname]))
df.head()

#print(df2.head())
# Get the Adjusted Close
df = df[['Adj Close']]
# Take a look at new data
#print(df.head())

Saving AML.L.csv to AML.L (1).csv


In [151]:
filename = os.path.basename(pathname)
filename_grab = os.path.splitext(filename)[0]

In [152]:
# A variable for predicting 'forecast_out' days out in future
forecast_out = 5

In [153]:
# Create another column (dependent variable) shifted 'forecast_out' units up
df['Prediction'] = df[["Adj Close"]].shift(-forecast_out)
#print(df.tail())

In [154]:
# Create the independent dataset
# Convert the dataframe to numpy array
X = np.array(df.drop(['Prediction'],1))
# Remove the last 'forecast_out' rows
X = X[:-forecast_out]
#print(X)

In [155]:
# Create the dependent dataset
# Convert the dataframe to numpy array (All of the values including the NaNs)
y = np.array(df['Prediction'])
# Get all of the y values except the last 'forecast_out' rows
y = y[:-forecast_out]
#print(y)

In [156]:
# Split the data into 80% training and 20% testing
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
# print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)
# train_set = x_train + y_train
# test_set = x_test + y_test

In [157]:
X, y = make_classification(n_samples=100, n_features=20, n_informative=15, n_redundant=5, random_state=1)
cv = KFold(n_splits=10, random_state=1, shuffle=True)

In [158]:
#Set x_forecast equal to the last forecast_out rows of the original dat set from Adj. Close column
x_forecast = np.array(df.drop(['Prediction'],1))[-forecast_out:]
#print(x_forecast)

In [159]:
ann_relu = Sequential()
ann_relu.add(Dense(200, input_dim=1, activation='relu'))
ann_relu.add(Dense(200, input_dim=200, activation='relu'))
ann_relu.add(Dense(200, input_dim=200, activation='relu'))
ann_relu.add(Dense(1, activation='linear'))

In [160]:
keras.optimizers.Adam(lr=0.1, beta_1=0.9, beta_2=0.999, amsgrad=False)
ann_relu.compile(loss='mean_squared_error', optimizer='RMSprop', metrics=['mean_absolute_percentage_error'])

In [161]:
ann_relu.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 200)               400       
_________________________________________________________________
dense_1 (Dense)              (None, 200)               40200     
_________________________________________________________________
dense_2 (Dense)              (None, 200)               40200     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 201       
Total params: 81,001
Trainable params: 81,001
Non-trainable params: 0
_________________________________________________________________


In [ ]:
history = ann_relu.fit(x_train, y_train, epochs=200, batch_size=32,validation_split=0.15,verbose=1)

In [163]:
y_pred_ann = ann_relu.predict(x_test)

In [164]:
mse_ann = mean_squared_error(y_test, y_pred_ann)
print(f'The MSE for the ANN algorithm is: {mse_ann}')

The MSE for the ANN algorithm is: 247578.35070433863


In [165]:
rmse_ann = mean_squared_error(y_test, y_pred_ann, squared=False)
print(f'The RMSE for the ANN algorithm is: {rmse_ann}')

The RMSE for the ANN algorithm is: 497.57245774292875


In [166]:
mae_ann = mean_absolute_error(y_test, y_pred_ann)
print(f'The MAE for the ANN algorithm is: {mae_ann}')

The MAE for the ANN algorithm is: 311.20539614905437


In [167]:
def MAPE_ANN(y_actual, y_prediction):
    mape = np.mean(np.abs((y_actual - y_prediction)/y_test))
    return mape

In [168]:
mape_ann = MAPE_ANN(y_test, y_pred_ann)
print(f'The MAPE for the ANN algorithm is: {mape_ann}')

The MAPE for the ANN algorithm is: 1.0366540845524748


In [169]:
n_array = np.array([])
new_array = np.append(n_array, [mse_ann, rmse_ann, mae_ann, mape_ann])
print(new_array)

[2.47578351e+05 4.97572458e+02 3.11205396e+02 1.03665408e+00]


In [170]:
new_df = pd.DataFrame(new_array)

In [171]:
filepath = f'{filename_grab}_ann_results.xlsx'
new_df.to_excel(filepath, index=False)

In [172]:
keras.backend.clear_session()